In [ ]:
import warnings
import pandas as pd
pd.options.plotting.backend = 'plotly'
warnings.filterwarnings('ignore')

In [ ]:
# Values in 1952, queried from the autumn database, hard-coded
death_rates = pd.Series(
    [0.048912, 0.005655, 0.003052, 0.009387, 0.0141716], 
    index=[0, 5, 15, 35, 50]
)

In [ ]:
upper_age = 100
ages = range(upper_age)

In [ ]:
# Get the death rates applicable to each age
life_table = pd.DataFrame(index=ages, columns=['death_rate'])
for a in ages:
    idx = next((i for i, age in enumerate(death_rates.index) if age > a), 0)  # The zero is a trick to get the last element when one is subtracted
    life_table.loc[a, 'death_rate'] = death_rates.iloc[idx - 1]

In [ ]:
df = life_table.reset_index()
df = df.rename(columns={'index': 'age', 'death_rate': 'qx'}) 
# qx: is the probability that a person aged exactly x dies before exact age (x+1), i.e., the probability of dying in the next year. Hence: qx= dx / lx
# lx: is the number of persons alive aged x in a life table.
df['lx'] = 100000
# dx: is the number of persons who die aged x last birthday.
for i in range(1, len(df)):
    df.loc[i, 'lx'] = df.loc[i - 1, 'lx'] - (df.loc[i - 1, 'qx'] * df.loc[i - 1, 'lx'])
    df["dx"] = 0
for x in range(100):
    if x >=99:
        df["dx"][x] = df["lx"][x]
    else:
        df["dx"][x] = df["lx"][x]-df["lx"][x+1]
#μx: is the force of mortality, i.e., represents the instantaneous rate at which people are dying.
df["μx"] = 0
df["μx"][0] = 2.0692602739726 * df["dx"][0] /(2*df["lx"][0])
for x in range(1,100):
    df["μx"][x] = (df["dx"][x-1]+df["dx"][x])/(2*df["lx"][x])

#Tx: is the total years of life to be lived by those aged exactly x (not the random variable Tx) until they all die
df["Tx"] = 0.000
for x in range(100):
    df["Tx"][x] = 0
    for y in range(x,100):
        df["Tx"][x] += df["lx"][y]
# e0x: is the complete expectation of life, i.e., the expectation of life allowing for the whole of the lifetime of the individual (years and days).
df["e0x"] = df["Tx"]/df["lx"] - 0.5
# total life expectancy, Lx: is the number of years of life lived between ages x and (x+1) of those currently aged x.
df["Lx"] = 0.000
for x in range(100):
    df["Lx"][x] = df["Tx"][x]-df["lx"][x]
# ex: is the curtate expectation of life, i.e., the expectation of life where only complete years of life count.
df["ex"] = df["Lx"]/df["lx"]
df